# Opis
Ten notes zawiera funkcje do pobierania danych. Najpierw wyciągane są id dokumentów, następnie pobierane dane rdf dla każdego dokumentu i wreszcie dame łączone są w jeden graf wiedzy.
## Opisy poszczególnych funkcji
- get_ids(file)
    
    Wyciąga id dokumentów z podanego pliku w standardzie RIS. Używa do tego lini z adresem url - tag "UR" np. `UR  - http://jbc.bj.uj.edu.pl/dlibra/publication/edition/510136`.

    Wymagane parametry:
    - **file** - ścieżka do pliku zawierającego dane w standardzie RIS

- get_rdfs(ids)

    Pobiera dokumenty rdf według podanych id.

    Wymagane parametry:
    - **ids** - lista id obiektów w JBC

- def save_rdfs_to_file(rdfs, ids, part)

    Zapisuje rdfy do plików w folerze `/data/partX`, gdzie X jest numerem określonym parametrem *part*.

    Wymagane parametry:
    - **rdfs** - lista pobranych danych o obiektach w formacie RDF
    - **ids** - lista id obiektów w JBC odpowiadającym podanym danym RDF
    - **part** - numer partii danych

In [6]:
from rdflib import Graph
import re       # regular expressions
import requests

SEARCH_URL = "https://jbc.bj.uj.edu.pl/dlibra/results?q=&action=SimpleSearchAction&type=-6&qf1=collections%3A188&qf2=collections%3A201&qf3=Subject%3Aspo%C5%82ecze%C5%84stwo&qf4=Subject%3Adruki%20ulotne%2020%20w.&qf5=Subject%3Adruki%20ulotne%2019%20w.&ipp=50"
    # parametr, które można dodać: "&ipp=50" to liczba wyników na stronie (50 tu, domyślnie jst 25), a "&p=0" oznacza numer strony (pierwsza ma nr 0)
RDF_URL = "https://jbc.bj.uj.edu.pl/dlibra/rdf.xml?type=e&id="

In [7]:
def get_ids(file):

    ids = []
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith("UR"):
                match = re.search("/edition/(\d+)", line)
                if match:
                    ids.append(match.group(1))
    print(f"Znaleziono {len(ids)} ID.")
    return ids

In [8]:
file = ".\data_ris\dlibra.ris"
ids = get_ids(file)

print(ids[0:10])

Znaleziono 150 ID.
['510136', '510137', '577130', '257952', '603761', '603763', '603767', '550767', '550793', '550799']


In [34]:
def get_rdfs(ids):
    rdfs = []
    for id in ids:
        print(f"Pobieram rdf dla id={id}")
        r = requests.get(RDF_URL+str(id))
        if r.ok:
            rdfs.append(r.content)
    print(f"Pobrano {len(rdfs)} rdfów.")
    return rdfs

In [ ]:
rdfs = get_rdfs(ids)
print(rdfs[0:10])
# pobranie 10 rdfów zajęło mi ok. 80 sekund
# pobranie 150 rdfów zajęło mi ok. 21 minut

In [36]:
def save_rdfs_to_file(rdfs, ids, part):
    for i, id in enumerate(ids):
        with open(f"./data/part{part}/{id}.rdf", "wb") as f:
            f.write(rdfs[i])

In [38]:
save_rdfs_to_file(rdfs, ids, 1)
